### Configurações do google colab

In [0]:
import os
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
path = 'gdrive/My Drive/Colab Notebooks/Secao 44  - Deep Learning - Exemplo de Caso de Negocios'

In [0]:
os.chdir(path)

In [5]:
!ls

AudioBooks_data_test.npz   AudioBooks_data_validation.npz  original.csv
AudioBooks_data_train.npz  model.ipynb			   pre-processed.ipynb


## Import the libreries

In [0]:
import numpy as np
from sklearn import preprocessing

In [0]:
raw_csv_data = np.loadtxt('original.csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:, -1]

## Balance the dataset

In [0]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equals_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equals_priors = np.delete(targets_all, indices_to_remove, axis=0)

## Standardize the inputs

In [0]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equals_priors)

## Shuffle the data

In [0]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equals_priors[shuffled_indices]

## Split the dataset into train, validation and test

In [11]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets) , ' | ' , train_samples_count , ' | ' , np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets) , ' | ' , validation_samples_count , ' | ' , np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets) , ' | ' , test_samples_count , ' | ' , np.sum(test_targets) / test_samples_count)

1791.0  |  3579  |  0.5004191114836547
221.0  |  447  |  0.49440715883668906
225.0  |  448  |  0.5022321428571429


## Save the three datasets in *.npz

In [0]:
np.savez('AudioBooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('AudioBooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('AudioBooks_data_test', inputs=test_inputs, targets=test_targets)